In [129]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [130]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [131]:
def getInfo(filesGrp, img_path):

  dslabel = {
    "id":"",
    "originalFileJPG":"",
    "originalFileGrp":[],
    "GeoTagInfo":{
      "coords":[],
      "datetime_original":"",
      "gps_altitude":0,
      "gps_altitude_ref":""    
    },
    "dest":{
      "rgb":"",
      "ndvi":"",
      "thumb":"",
    },
    "info":{
      "course":"",
      "area":"",
      "desc":"",
      "alt":""
    },
    "label":[],
    "annotation":[]
  }

  

  dslabel['originalFileJPG'] = img_path[3:]
  dslabel['originalFileGrp'] = [x[3:] for x in filesGrp  if ('.JPG' not in x)]

  with open(img_path, 'rb') as src:
    img = exifimg(src)

  coords = [
    decimal_coords(img.gps_longitude,
    img.gps_longitude_ref),
      decimal_coords(img.gps_latitude,
    img.gps_latitude_ref)
  ]

  dslabel['id'] = img.datetime_original.replace(" ","").replace(":","") + "_{:.10f}".format(coords[0]).replace(".","") + "_{:.10f}".format(coords[1]).replace(".","") + "_{:f}".format(img.gps_altitude).replace(".","")
  dslabel['GeoTagInfo']['coords'] = coords
  dslabel['GeoTagInfo']['datetime_original'] = img.datetime_original
  dslabel['GeoTagInfo']['gps_altitude'] = img.gps_altitude
  dslabel['GeoTagInfo']['gps_altitude_ref'] = img.gps_altitude_ref
  
  return dslabel

In [132]:
def getNDVIimg(filesGrp):


  with rasterio.open([ x for x in filesGrp if ('_MS_R' in x) ][0]) as src:
      band_red = src.read(1).astype(float)/65536.

  with rasterio.open([ x for x in filesGrp if ('_MS_NIR' in x) ][0]) as src:
      band_nir = src.read(1).astype(float)/65536.

  np.seterr(divide='ignore', invalid='ignore')

  # Calculate NDVI
  ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float))

  ndvi -= ndvi.min() # ensure the minimal value is 0.0
  ndvi /= ndvi.max() # maximum value in image is now 1.0

  cm = plt.cm.get_cmap('RdYlGn')
  ndvi_cm = cm(ndvi)

  ndvi_img = cv2.normalize(ndvi_cm[:,:,:3], None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8)

  return ndvi_img

In [133]:
def getalignedRGB(filesGrp_, ndvi_img):
  RGBimg = np.asarray(Image.open(filesGrp_[0]))

  # RGBimg = np.asarray(Image.open(filesGrp_[0]).resize((ndvi_img.shape[1],ndvi_img.shape[0])))
  # features0 = FeatureExtraction(RGBimg)
  # features1 = FeatureExtraction(ndvi_img)

  # matches = feature_matching(features0, features1)

  # print('{} number of matched has been found! in {}'.format(len(matches), filesGrp_[0]))

  # if len(matches) < 10:
  #    raise Exception("Not Enough Matches")

  # H, _ = cv2.findHomography( features0.matched_pts, features1.matched_pts, cv2.RANSAC, 5.0)

  H = np.array([[ 5.93209530e-01, -8.35074813e-03, -2.21474434e+02],
       [ 1.32198817e-02,  5.96460559e-01, -2.01042419e+02],
       [ 1.58520522e-06,  1.69424781e-06,  1.00000000e+00]])

  h, w, c = ndvi_img.shape
  RGBimg_tuned = cv2.warpPerspective(RGBimg, H, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

  return RGBimg_tuned 

In [165]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS'

course_list = ['청도','포항','중문']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
}

folder_type = ['rgb','ndvi','thumb']

In [170]:
course_ids.get('포항')

'MGC001'

In [171]:
subFolder = glob.glob(os.path.join(img_folder,'*'))
To_create_folder = [os.path.join(out_folder, list(filter(lambda y: y in x, course_list))[0],os.path.split(x)[-1][:9].replace('_','') ) for x in subFolder]

In [175]:
subFolder

['D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완']

In [172]:
To_create_folder

['D:\\ToAWS\\포항\\20230811']

In [173]:
import itertools

iterables = [ To_create_folder, folder_type]

paths_ = [os.path.join(x[0],x[1]) for x in list(itertools.product(*iterables))]

# for path_ in paths_:
#   if not os.path.exists(path_):
#     os.makedirs(path_)

In [174]:
paths_

['D:\\ToAWS\\포항\\20230811\\rgb',
 'D:\\ToAWS\\포항\\20230811\\ndvi',
 'D:\\ToAWS\\포항\\20230811\\thumb']

In [188]:
files = glob.glob(os.path.join(img_folder,'**/DJI_*.JPG'), recursive=True)

len(files)

7571

In [189]:
files[0]

'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해 4H\\2023_0811_포항CC_동해 4H_0.5m\\20230811_1128-1135_물리적피해,음지피해,켄터키,티,잡초,이종잔디,켄터키,패취류,썸머패취\\DJI_20230811112755_0446_D.JPG'

In [190]:
os.path.normpath(files[0]).split(os.sep)

['D:',
 'original',
 '2023_0811_포항CC_비온다음날_데이터 1차 분류_완',
 '동해 4H',
 '2023_0811_포항CC_동해 4H_0.5m',
 '20230811_1128-1135_물리적피해,음지피해,켄터키,티,잡초,이종잔디,켄터키,패취류,썸머패취',
 'DJI_20230811112755_0446_D.JPG']

In [197]:
pathlist = []

for idx, file_ in enumerate(files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  course_id = course_ids.get(course)
  target_date = os.path.split(file_)[-1].split('_')[1][:8]
  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3]

  if len(info[5].split('_')) ==3 :
    desc = info[5].split('_')[-1]
  else:
    desc = "미기재"

  alt =  info[4].split('_')[-1]

  # print(out_folder,course,target_date,area,desc,'rgb')
  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'rgb')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'thumb')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = ('/').join([target_date,area,desc])

  if path_ not in pathlist:
    pathlist.append(path_)


In [198]:
pathlist

['20230811/동해 4H/물리적피해,음지피해,켄터키,티,잡초,이종잔디,켄터키,패취류,썸머패취',
 '20230811/동해 4H/물리적피해,플레이어답압',
 '20230811/동해 4H/물리적피해,음지피해,잡초,이종잔디,패취류,썸머패취',
 '20230811/동해 4H/물리적피해,과습피해,플레이어답압,토양병,페어리링',
 '20230811/동해 4H/미기재',
 '20230811/동해 5H/물리적피해,건조피해,토양병,페어리링,물리적피해,과습피해',
 '20230811/동해 5H/물리적피해,건조피해,과습피해,토양병,페어리링',
 '20230811/동해 5H/미기재',
 '20230811/동해 6H/미기재',
 '20230811/동해 6H/물리적피해,디봇,플레이어답압,잡초,사초과,파대가리',
 '20230811/동해 6H/토양병,페어리링,물리적피해,플레이어답압',
 '20230811/동해 6H/물리적피해,플레이어답압,디봇',
 '20230811/동해_2H/물리적피해,플레이어답압,한국잔디,티',
 '20230811/동해_2H/잡초,사초과,방동사니',
 '20230811/동해_2H/토양병,페어리링,벤트그래스,그린,물리적피해,장비피해,한국잔디,러프',
 '20230811/동해_2H/물리적피해,플레이어답압',
 '20230811/동해_2H/장비피해,잡초,사초과,방동사니',
 '20230811/동해_2H/물리적피해,장비피해',
 '20230811/동해_2H/토양병,페어리링',
 '20230811/동해_2H/미기재',
 '20230811/동해_3H/잡초,화본과,바랭이,잡초,이종잔디,벤트,잡초,광엽,피막이',
 '20230811/동해_3H/잡초,화본과,띠',
 '20230811/동해_3H/토양병,페어리링,한국잔디,티',
 '20230811/동해_3H/잡초,이종잔디,켄터키,토양병,페어리링,한국잔디,티,잡초,화본과,바랭이',
 '20230811/동해_3H/장비피해,페어리링',
 '20230811/동해_3H/물리적피해,기타피해,벤트그래스,그린,토양병,페어리링,벤트그래스,그린',

In [178]:
filenameOnly = []
for file_ in files:
  filenameOnly.append(os.path.split(file_)[-1])

In [180]:
# target_files = [ x for x in files if ('1.5m'in x or '1.5M'in x ) ][0:500] + [ x for x in files if ('0.5m'in x or '0.5M'in x) ][100:600] + [ x for x in files if ('30m' in x or '30M' in x) ][0:100]
target_files = files
len(target_files)

7571

In [182]:
path = os.path.normpath(file_)
path.split(os.sep)

['D:',
 'original',
 '2023_0811_포항CC_비온다음날_데이터 1차 분류_완',
 '동해_3H',
 '2023_0811_포항CC_동해 3H_30m',
 '20230811_1057-1103',
 'DJI_20230811110326_0766_D.JPG']

In [181]:
glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ os.path.split(file_)[1].split('_')[1][:-2]+"**_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)

['D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해_3H\\2023_0811_포항CC_동해 3H_30m\\20230811_1057-1103\\DJI_20230811110326_0766_D.JPG',
 'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해_3H\\2023_0811_포항CC_동해 3H_30m\\20230811_1057-1103\\DJI_20230811110326_0766_MS_G.TIF',
 'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해_3H\\2023_0811_포항CC_동해 3H_30m\\20230811_1057-1103\\DJI_20230811110326_0766_MS_NIR.TIF',
 'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해_3H\\2023_0811_포항CC_동해 3H_30m\\20230811_1057-1103\\DJI_20230811110326_0766_MS_R.TIF',
 'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해_3H\\2023_0811_포항CC_동해 3H_30m\\20230811_1057-1103\\DJI_20230811110326_0766_MS_RE.TIF']

In [164]:
InfoJsonArray = []

for idx, file_ in enumerate(target_files):

  course = list(filter(lambda y: y in file_, course_list))[0]
  target_date = os.path.split(file_)[-1].split('_')[1][:8]
  path = os.path.normpath(file_)
  info = path.split(os.sep)

  area = info[3].split('_')[-1]
  if len(info[5].split('_')) ==3 :
    desc = info[5].split('_')[-1]
  else:
    desc = "미기재"
  alt =  info[4].split('_')[-1]

  filesGrp = glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ os.path.split(file_)[1].split('_')[1][:-3]+"***_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)
  # print(filesGrp)
  file_info = getInfo(filesGrp, file_)

  file_info['info']['course'] = course
  file_info['info']['area'] = area
  file_info['info']['alt'] = alt
  file_info['info']['desc'] = desc
  file_info['info']['date'] = target_date


  # try:
  ndvi_img = getNDVIimg(filesGrp)
  rgb_img = getalignedRGB(filesGrp, ndvi_img)
  resized_ndvi = cv2.resize(ndvi_img, dsize=(648,486), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi','ndvi{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_ndvi)
  im.save(save_name)
  file_info['dest']['ndvi'] = save_name[9:]

  resized_rgb = cv2.resize(rgb_img , dsize=(2592,1944), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'rgb','rgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(resized_rgb)
  im.save(save_name)
  file_info['dest']['rgb'] = save_name[9:]


  thumb_rgb = cv2.resize(rgb_img , dsize=(324,243), interpolation=cv2.INTER_CUBIC)
  save_name = os.path.join(out_folder,course_id,target_date,area,desc,'thumb','thumbrgb{}.JPG'.format( file_info['id']) )
  im = Image.fromarray(thumb_rgb)
  im.save(save_name)
  file_info['dest']['thumb'] = save_name[9:]

  # except Exception:
  #   pass  # or you could use 'continue'





  InfoJsonArray.append( file_info)

  
  print(save_name)

  print('{} out of {} files Completed'.format(idx, len(target_files)))



  

D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162119_1286447212222_356634586944_159912000.JPG
0 out of 8320 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162121_1286447209722_356634581111_159996000.JPG
1 out of 8320 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162123_1286447206111_356634575278_159939000.JPG
2 out of 8320 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162125_1286447208889_356634552222_159989000.JPG
3 out of 8320 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162127_1286447220000_356634515556_160034000.JPG
4 out of 8320 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162129_1286447220278_356634490278_160052000.JPG
5 out of 8320 files Completed
D:\ToAWS\MGC002\20230823\스타트퍼팅그린\엽고성병,달라스팟,벤트,그린\thumb\thumbrgb20230823162131_1286447228611_356634475556_1

In [155]:
save_name

'D:\\ToAWS\\MGC002\\20230823\\밸리6H\\미기재\\thumb\\thumbrgb20230823105129_1286466081667_356525362222_185064000.JPG'

In [163]:
filesGrp

['D:\\original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\1_밸리6H\\2023_0823_청도그레이스_밸리6H_1.5m\\20230823_1054-1116_건조,답압,장비피해,조류\\DJI_20230823111359_0531_D.JPG']

In [156]:
len(InfoJsonArray)

1100

In [157]:
save_name = os.path.join(out_folder,course_id,target_date, 'data.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)

In [158]:
with open('mydata.json', "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)